In [1]:
from json import dumps, loads
import time, os
import threading, Queue
import ast
import unittest
import requests
import random
import logging
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient

# Setup logging
logging.basicConfig(level=logging.INFO, filename="execution.log", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
logging.info("Starting test_surge_bids")

# Setup variables
r = requests.get('http://10.129.6.52:4999/setup.in')
setup_str = r.text
setup_str = setup_str.split('\n')
NUM_SHARDS = int(setup_str[0])
NUM_REPLICAS = int(setup_str[1])
NUM_CLIENTS = int(setup_str[2])
logging.info("NUM_SHARDS: %s, NUM_REPLICAS: %s, NUM_CLIENTS: %s "%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS))
CS_HOST='10.129.6.52'
G = setup()[0]


In [2]:
global_client = ChainspaceClient(host=CS_HOST,port=5000)
init_transaction = surge.init()
init_tokens = init_transaction['transaction']['outputs']
global_client.process_transaction(init_transaction)
client_divs = surge.eq_div(NUM_CLIENTS, NUM_SHARDS)
print client_divs

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken

In [3]:
# Create prosumer clients for locations 
clients = []
base_port=5000
sreps = []

for s in range(0,NUM_SHARDS):
    clients.append([])
    sreps.append(surge.SREPClient(host=CS_HOST, port=base_port+s))
    
    for c in range(0,client_divs[s]):
        clients[s].append(surge.SurgeClient(host=CS_HOST, port=base_port+s))

print clients

threads = []
for s in range(0,NUM_SHARDS):
    threads.append([])
    for c in range(0,client_divs[s]):
        client = clients[s][c]
        idx = sum(client_divs[:s]) + c
        threads[s].append(threading.Thread(target=clients[s][c].create_surge_client, args=(init_tokens[idx],)) )

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()

[[<chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7dd0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7cf8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7e18>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7e60>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7ea8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7ef0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7f38>], [<chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007a7fc8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007ab050>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007ab0e0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007ab098>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007ab128>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7fba007ab170>, <chainsp

POST http://10.129.6.52:5002/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["e6f0957aad0a58104f8a7ffbc011c6d5a6702948fbdf12b82231557a7e40939e"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd037cedf7a64a0200c04a3daddd5d0a1723fa4dcecf6c4aea54270e2316", "92c71d002bb1ad25a2350d53b002edfe93f41491a5f97687859ed29d3b7df39c2ec71d002b21876bd0d9b7b022edc574f110584d79516533d383705d925cdbfc3a"], "outputs": ["{\"timestamp\": 1581333887.187052, \"type\": \"SurgeClient\", \"location\": 2, \"pub\": \"c7220292cd02c9bd037cedf7a64a0200c04a3daddd5d0a1723fa4dcecf6c4aea54270e2316\"}", "{\"timestamp\": 1581333887.187066, \"type\": \"VoteSlipToken\", \"location\": 2, \"pub\": \"c7220292cd02c9bd037cedf7a64a0200c04a3daddd5d0a1723fa4dcecf6c4aea54270e2316\"}", "{\"timestamp\": 1581333887.187101, \"type\": \"EBToken\", \"location\": 2, \"pub\": \"c7220292cd02c9bd037cedf7a64a0200c04a3daddd5d0a1723fa4dcecf6c4aea54270e2316\"}"], "returns": [], "dependencies": [], "referenceInp

POST http://10.129.6.52:5002/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["a04ba17ec3a4f7d9a9ded4c9d4438f7283a43a8a453a76319d817f3c5d5840b0"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd0381443840c6ff245e994b9388b03ac17cf8b7b0ed2c4a894e94bfe1d5", "92c71d002b5c2f2dcdf72ab4e777e4012ddfc25f47126878e7430b190c11fcc1f0c71d002b0407eb5cdb5d40b080233ef0dfe933ca3ab9094d6bc19e2d815c9722"], "outputs": ["{\"timestamp\": 1581333887.171057, \"type\": \"SurgeClient\", \"location\": 2, \"pub\": \"c7220292cd02c9bd0381443840c6ff245e994b9388b03ac17cf8b7b0ed2c4a894e94bfe1d5\"}", "{\"timestamp\": 1581333887.171111, \"type\": \"VoteSlipToken\", \"location\": 2, \"pub\": \"c7220292cd02c9bd0381443840c6ff245e994b9388b03ac17cf8b7b0ed2c4a894e94bfe1d5\"}", "{\"timestamp\": 1581333887.171122, \"type\": \"EBToken\", \"location\": 2, \"pub\": \"c7220292cd02c9bd0381443840c6ff245e994b9388b03ac17cf8b7b0ed2c4a894e94bfe1d5\"}"], "returns": [], "dependencies": [], "referenceInp

POST http://10.129.6.52:5006/api/1.0/transaction/process HTTP/1.1POST http://10.129.6.52:5005/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["5074c6620926c4c8fa6278ec58a29c0ea3b248d1db466c2fdcf8c703745c7634"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd031545630c20b26308e7981cefdce4d2acc87cb5f4e482bc5ac53e7dce", "92c71d002b64dd2a9565a0f9b5280b430a59623dd72a5fbf085e80f71491b76a8ec71d002b02e1407eb74de8ba75eec27e8c9b5ec4fe0518d86e5e2d50e1ebbfba"], "outputs": ["{\"timestamp\": 1581333887.405263, \"type\": \"SurgeClient\", \"location\": 5, \"pub\": \"c7220292cd02c9bd031545630c20b26308e7981cefdce4d2acc87cb5f4e482bc5ac53e7dce\"}", "{\"timestamp\": 1581333887.405276, \"type\": \"VoteSlipToken\", \"location\": 5, \"pub\": \"c7220292cd02c9bd031545630c20b26308e7981cefdce4d2acc87cb5f4e482bc5ac53e7dce\"}", "{\"timestamp\": 1581333887.405319, \"type\": \"EBToken\", \"location\": 5, \"pub\": \"c7220292cd02c9bd031545630c20b26308e7981cefdce4d2acc87cb5f4e482bc

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

In [4]:
# Create threads for casting srep vote

threads = []
queues = []

for s in range(0,NUM_SHARDS):
    que = Queue.Queue()
    threads.append([])
    for c in range(0,client_divs[s]):
        t = threading.Thread(target=clients[s][c].cast_srep_vote, args=(sreps[s].pub, que)) 
        threads[s].append(t)
    queues.append(que)
start = time.time()

# Run voting threads
for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()



POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["01b5ab93145a9e9f7a41266530d6f98723d85ff843de8de4980f7fc2d2d123c6"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b8d01461ca39f566c23289bd3393f7b8bc5660c99aea127d0c9f34fefc71d002be1cfce56ba931a2481234f203c3507fbb959a985f3eb149c662c1919"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138db45d9d9359dc4f39a60088f\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd035744033ac8aeb16a8b62bd2df09083df4657691415ed6b486ca445a5\", \"timestamp\": 1581333897.16554, \"location\": 0}", "{\"timestamp\": 1581333887.10422, \"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd035744033ac8aeb16a8b62bd2df09083df4657691415ed6b486ca445a5\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"01b5ab93145a9e9f7a41266530d6f98723d85ff843de8de4980f7fc2d2d123c6": "{\"timestamp\": 1581333887.10422

POST http://10.129.6.52:5001/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["88ec69d82f4dbd03d525fc5b08139ceae3505da0f0338392b6debe37af84787b"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b6625fee349bffe1acaecb76792bbde8c4935d5d98fb039b3f83d46d0c71d002bdef191fce46709c725d8b79d2d81c44339c6006add9e8a976fbb8b9b"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd020cc72dff7032da0dca3c93887c9a584ca8894195e9cffb806debfa06\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02ecab574655714b89e656b1a2c487d43f24885a2a7fe499106b3d7a08\", \"timestamp\": 1581333897.188608, \"location\": 1}", "{\"timestamp\": 1581333887.1225, \"type\": \"VoteSlipToken\", \"location\": 1, \"pub\": \"c7220292cd02c9bd02ecab574655714b89e656b1a2c487d43f24885a2a7fe499106b3d7a08\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"88ec69d82f4dbd03d525fc5b08139ceae3505da0f0338392b6debe37af84787b": "{\"timestamp\": 1581333887.1225,

POST http://10.129.6.52:5004/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["b437fbdf72810a8889900668685d235658acef465e35571f9982ff568f4849e2"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bbe47d66346a5adc4d201273edf8bbbf1dbbcdecc983f08f4ad522afec71d002bd55e649933975d3a65a1008bb868830d55baa41e218a44ed2655b122"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02634b59c4ea5eb4eafb9d25c0a18b96d8d8a5ec243f6128e929b1005e\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02b73ac1d1fc5b13d36afecf9c05e1aef643e05e477c1d66f2bb234d1e\", \"timestamp\": 1581333897.306618, \"location\": 4}", "{\"timestamp\": 1581333887.286927, \"type\": \"VoteSlipToken\", \"location\": 4, \"pub\": \"c7220292cd02c9bd02b73ac1d1fc5b13d36afecf9c05e1aef643e05e477c1d66f2bb234d1e\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"b437fbdf72810a8889900668685d235658acef465e35571f9982ff568f4849e2": "{\"timestamp\": 1581333887.286

POST http://10.129.6.52:5007/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["c3f28864aacede7e9f59a6c39dc41fdd099f184ea323550f28e08decf102a92b"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bfd041d065c6f596debab7756e2c27caaa93d374d135067a871ec3b2dc71d002b8e5115cbc59395fd7ce056592052c397487e2cc35229b757dd6312d5"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd03607f13ec3a911a72f3f18c8cfb1584c8a9a31510416899a94f2345db\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd0223e118244102cf834ae9352c0c15898dac31bd35ebb71c29c4b6cbb7\", \"timestamp\": 1581333897.93094, \"location\": 7}", "{\"timestamp\": 1581333887.670954, \"type\": \"VoteSlipToken\", \"location\": 7, \"pub\": \"c7220292cd02c9bd0223e118244102cf834ae9352c0c15898dac31bd35ebb71c29c4b6cbb7\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"c3f28864aacede7e9f59a6c39dc41fdd099f184ea323550f28e08decf102a92b": "{\"timestamp\": 1581333887.6709

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

In [5]:
# # Create srep without threading
# for s in range(0,NUM_SHARDS):
#     votes = []
#     que = queues[s]
#     while not que.empty():
# #     for i in range(0,2):
#         votes.append(que.get())
#     client = sreps[s]
#     sreps[s].create_srep_client(host=CS_HOST, srep_port=base_port+s, vote_tokens=tuple(votes))

In [6]:
# Create srep using threading
threads = []
for s in range(0,NUM_SHARDS):
    votes = []
    que = queues[s]
    while not que.empty():
        votes.append(que.get())
#     client = sreps[s]
    t = threading.Thread(target=sreps[s].create_srep_client, args=(CS_HOST, base_port, tuple(votes)) )
    threads.append(t)

create_start = time.time()
    
for t in threads:
    t.start()
    
for t in threads:
    t.join()

end = time.time()
duration = end-start
create_duration = end-create_start
logging.info("Execution took "+ str(duration))
logging.info("Multi-input txns took "+ str(create_duration))
logging.info("TPS: %s"%((NUM_CLIENTS+NUM_SHARDS)/duration))
logging.info("Multi-input TPS: %s"%(NUM_SHARDS/create_duration))
with open('stats.csv','a') as f:
    f.write("%s, %s, %s, %s, %s\n"%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS, (NUM_CLIENTS+NUM_SHARDS)/duration, NUM_SHARDS/create_duration))


POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["cad37e0a89a945d2546138ef90d057ff23b19b444ec70488b0e5a7c5df33bd00", "8f79d2ebcb7918eb6cab26f8c5ef6256b10375f3c08374b4a8d2850d1b74eba7", "33db4da88b98fe854c38006b77a27a6e0a5e02da17f77849704af6d9d7626d88", "73e2ce92b01ff2a164331dab4cc69480ce3f3eaf1845951b8873b06f1f24364b", "f2c99804300523291ce157134fddd2c188c98891af70133ab871475ec388673c", "b509aa6eeec2d225cf1d4f073a8f1f063d8ef7dd7b3a546f7c0235ec5011b1c8", "8a3579af0245239a1e6e0f0df31ee31053b199edbc2bc6ecfd50f0a5039e83c8"], "methodID": "create_srep_client", "parameters": ["c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138db45d9d9359dc4f39a60088f", "92c71d002b3751f85c6908cd21561d8026542599a3efbf4a31dde2528c870d3bb9c71d002ba80ad59c6b0aa8aca9097c4f2a416c0fe018ad93bc9f80809b22a85f"], "outputs": ["{\"timestamp\": 1581333901.326361, \"type\": \"SREPClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138db45d9d9359dc

Retrying...
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["cad37e0a89a945d2546138ef90d057ff23b19b444ec70488b0e5a7c5df33bd00", "8f79d2ebcb7918eb6cab26f8c5ef6256b10375f3c08374b4a8d2850d1b74eba7", "33db4da88b98fe854c38006b77a27a6e0a5e02da17f77849704af6d9d7626d88", "73e2ce92b01ff2a164331dab4cc69480ce3f3eaf1845951b8873b06f1f24364b", "f2c99804300523291ce157134fddd2c188c98891af70133ab871475ec388673c", "b509aa6eeec2d225cf1d4f073a8f1f063d8ef7dd7b3a546f7c0235ec5011b1c8", "8a3579af0245239a1e6e0f0df31ee31053b199edbc2bc6ecfd50f0a5039e83c8"], "methodID": "create_srep_client", "parameters": ["c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138db45d9d9359dc4f39a60088f", "92c71d002b3751f85c6908cd21561d8026542599a3efbf4a31dde2528c870d3bb9c71d002ba80ad59c6b0aa8aca9097c4f2a416c0fe018ad93bc9f80809b22a85f"], "outputs": ["{\"timestamp\": 1581333901.326361, \"type\": \"SREPClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138d

Retrying...
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["cad37e0a89a945d2546138ef90d057ff23b19b444ec70488b0e5a7c5df33bd00", "8f79d2ebcb7918eb6cab26f8c5ef6256b10375f3c08374b4a8d2850d1b74eba7", "33db4da88b98fe854c38006b77a27a6e0a5e02da17f77849704af6d9d7626d88", "73e2ce92b01ff2a164331dab4cc69480ce3f3eaf1845951b8873b06f1f24364b", "f2c99804300523291ce157134fddd2c188c98891af70133ab871475ec388673c", "b509aa6eeec2d225cf1d4f073a8f1f063d8ef7dd7b3a546f7c0235ec5011b1c8", "8a3579af0245239a1e6e0f0df31ee31053b199edbc2bc6ecfd50f0a5039e83c8"], "methodID": "create_srep_client", "parameters": ["c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138db45d9d9359dc4f39a60088f", "92c71d002b3751f85c6908cd21561d8026542599a3efbf4a31dde2528c870d3bb9c71d002ba80ad59c6b0aa8aca9097c4f2a416c0fe018ad93bc9f80809b22a85f"], "outputs": ["{\"timestamp\": 1581333901.326361, \"type\": \"SREPClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138d

Retrying...
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["cad37e0a89a945d2546138ef90d057ff23b19b444ec70488b0e5a7c5df33bd00", "8f79d2ebcb7918eb6cab26f8c5ef6256b10375f3c08374b4a8d2850d1b74eba7", "33db4da88b98fe854c38006b77a27a6e0a5e02da17f77849704af6d9d7626d88", "73e2ce92b01ff2a164331dab4cc69480ce3f3eaf1845951b8873b06f1f24364b", "f2c99804300523291ce157134fddd2c188c98891af70133ab871475ec388673c", "b509aa6eeec2d225cf1d4f073a8f1f063d8ef7dd7b3a546f7c0235ec5011b1c8", "8a3579af0245239a1e6e0f0df31ee31053b199edbc2bc6ecfd50f0a5039e83c8"], "methodID": "create_srep_client", "parameters": ["c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138db45d9d9359dc4f39a60088f", "92c71d002b3751f85c6908cd21561d8026542599a3efbf4a31dde2528c870d3bb9c71d002ba80ad59c6b0aa8aca9097c4f2a416c0fe018ad93bc9f80809b22a85f"], "outputs": ["{\"timestamp\": 1581333901.326361, \"type\": \"SREPClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138d

Retrying...
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["cad37e0a89a945d2546138ef90d057ff23b19b444ec70488b0e5a7c5df33bd00", "8f79d2ebcb7918eb6cab26f8c5ef6256b10375f3c08374b4a8d2850d1b74eba7", "33db4da88b98fe854c38006b77a27a6e0a5e02da17f77849704af6d9d7626d88", "73e2ce92b01ff2a164331dab4cc69480ce3f3eaf1845951b8873b06f1f24364b", "f2c99804300523291ce157134fddd2c188c98891af70133ab871475ec388673c", "b509aa6eeec2d225cf1d4f073a8f1f063d8ef7dd7b3a546f7c0235ec5011b1c8", "8a3579af0245239a1e6e0f0df31ee31053b199edbc2bc6ecfd50f0a5039e83c8"], "methodID": "create_srep_client", "parameters": ["c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138db45d9d9359dc4f39a60088f", "92c71d002b3751f85c6908cd21561d8026542599a3efbf4a31dde2528c870d3bb9c71d002ba80ad59c6b0aa8aca9097c4f2a416c0fe018ad93bc9f80809b22a85f"], "outputs": ["{\"timestamp\": 1581333901.326361, \"type\": \"SREPClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03f0c7104b9df54a5cc555ef52a4afd138d

HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'SUBMIT_T_SYSTEM_ERROR', u'success': u'False'}


In [7]:
# end = time.time()
# duration = end-start
# logging.info("Execution took "+ str(duration))
# logging.info("TPS: %s"%(NUM_SHARDS/duration))

# with open('stats.csv','a') as f:
#     f.write("%s, %s, %s, %s\n"%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS, NUM_SHARDS/duration))
sreps[7].create_srep_client(CS_HOST, base_port, tuple(votes))

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["90cb25326c9f40b7f0123e888fb948d64e1346f2ab59b8207e8ba5a4e29c3846", "5ca840d116ae4c79ae15291842ab0f536231d3664d4821a36bb87ec03a787787", "be69f34d00c3bbe4b4cc12dde910fabd570c2cab6aff8543d5e2716f07584048", "16f704b7247098f72d9dd81efd2bb4111fee7ee8766fec7ebfff4288b21b7479", "1045f983f8c72a714309a39d56f912b1087255bf2936334276c60f5bf37b5719", "66c9fb1ca9b1867f440d0920c598873841080e7d0f81f9b3ecd2aef78ec2d4ff"], "methodID": "create_srep_client", "parameters": ["c7220292cd02c9bd03607f13ec3a911a72f3f18c8cfb1584c8a9a31510416899a94f2345db", "92c71d002bde446e0983999d20bd04498ca99476c8ed37c25987e19f2f2d879928c71d002b10a57d39f15706bde371d439e7ff53cc8a4766f9dab600fa0b9b5549"], "outputs": ["{\"timestamp\": 1581333953.241753, \"type\": \"SREPClient\", \"location\": 7, \"pub\": \"c7220292cd02c9bd03607f13ec3a911a72f3f18c8cfb1584c8a9a31510416899a94f2345db\"}", "{\"timestamp\": 1581333953.241817, \"type\": \"Vot

Retrying...
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["90cb25326c9f40b7f0123e888fb948d64e1346f2ab59b8207e8ba5a4e29c3846", "5ca840d116ae4c79ae15291842ab0f536231d3664d4821a36bb87ec03a787787", "be69f34d00c3bbe4b4cc12dde910fabd570c2cab6aff8543d5e2716f07584048", "16f704b7247098f72d9dd81efd2bb4111fee7ee8766fec7ebfff4288b21b7479", "1045f983f8c72a714309a39d56f912b1087255bf2936334276c60f5bf37b5719", "66c9fb1ca9b1867f440d0920c598873841080e7d0f81f9b3ecd2aef78ec2d4ff"], "methodID": "create_srep_client", "parameters": ["c7220292cd02c9bd03607f13ec3a911a72f3f18c8cfb1584c8a9a31510416899a94f2345db", "92c71d002bde446e0983999d20bd04498ca99476c8ed37c25987e19f2f2d879928c71d002b10a57d39f15706bde371d439e7ff53cc8a4766f9dab600fa0b9b5549"], "outputs": ["{\"timestamp\": 1581333953.241753, \"type\": \"SREPClient\", \"location\": 7, \"pub\": \"c7220292cd02c9bd03607f13ec3a911a72f3f18c8cfb1584c8a9a31510416899a94f2345db\"}", "{\"timestamp\": 1581333953.241817, \"t